In [1]:
import pandas as pd
import janitor
import numpy as np
from IPython.display import display

import sys

sys.path.append("/home/lsys/pwned_pols/venv/lib/python3.10/site-packages")

from utilities import (
    clean_dedupe_email_column,
    pandas_to_tex,
    #     clean_email_column_no_dedupe,
    classify_comm_gov_email,
)
from utilities import LIST_SERIOUS_DATACLASSES

pd.set_option("display.max_columns", 150)
pd.set_option("display.max_rows", 150)
pd.set_option("display.max_colwidth", None)

import warnings

warnings.filterwarnings("ignore")

## Prep EP data

In [2]:
df_ep_emails = (
    pd.read_csv(
        "../data/everypol/everypol_combined_legislature_data.csv", low_memory=False
    )
    .sort_values(["cc3", "leg_start_year", "email"])
    .pipe(lambda df_: clean_dedupe_email_column(df_))
    .drop_duplicates(subset=["email"], keep="first", ignore_index=True)
    # ================================================================
    # Fix missing cc3 for Wales/Scotland
    # Wales, Scotland = GBR
    .assign(
        cc3=lambda df_: np.where(
            df_["cc"] == "GB-SCT",
            "GBR",
            np.where(df_["cc"] == "GB-WLS", "GBR", df_["cc3"]),
        )
    )
    # ================================================================
    # Fix missing pop for Wales/Scotland
    .assign(
        pop2024=lambda df_: np.where(
            df_["cc"] == "GB-SCT",
            "GBR",
            np.where(df_["cc"] == "GB-WLS", 68556800, df_["pop2024"]),
        )
    )
    # ================================================================
    # Fix ltype for Namibia
    .assign(ltype=lambda df_: np.where((df_["cc3"]=="NAM") & (df_["legislature"]=="National Assembly"), "lower house", df_["ltype"]))
    # ================================================================
    # Fix ltype for India to bicameral
    .assign(
        ltype=lambda df_: np.where(
            df_["cc3"] == "IND", "lower house", df_["ltype"]
        )
    )
    # ================================================================
    # Get #emails per country (cc3)
    .assign(nemail_cc3=lambda df_: df_.groupby("cc3")["email"].transform("nunique"))
    # ================================================================
    .reorder_columns(
        [
            "email",
            "cc3",
            "country",
            "ltype",
            "legislature",
            "chamber",
            "leg_start_year",
            "nemail_cc3",
            "gender",
        ]
    )
)

assert (df_ep_emails["nemail_cc3"] >= 30).all()
display(df_ep_emails.head(3))
df_ep_emails.info()

,email,cc3,country,ltype,legislature,chamber,leg_start_year,nemail_cc3,gender,id,name,sort_name,twitter,facebook,group,group_id,area_id,area,term_x,start_date,end_date,image,wikidata,wikidata_group,wikidata_area,row_id,term_y,row_count,n_unique_emails,person_count_legistype,url,cc,leg_start_date,lastmod,pop2024,lastmod_year,domain
0,albana.vokshi@parlament.al,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,female,c8bd71fc-4815-40b5-a5c4-bd359e3b0cef,Albana Vokshi,VOKSHI ALBANA,NaN,NaN,PD,c26946f7-8c5e-4474-a85e-ee5342cf4006,durrës_county,Durrës County,7,NaN,NaN,https://www.parlament.al/wp-content/uploads/2015/12/Albana-Vokshi-PD.jpg,Q4709025,Q845743,NaN,490,VII Pluralist Legislature,140,31,292,https://cdn.rawgit.com/everypolitician/everypolitician-data/4ace3b1548b95eb669deab4d99bbc894639269f2/data/Albania/Assembly/term-7.csv,AL,2009-09-08,1557734928,2740502.0,2019,parlament.al
1,aldo.bumci@parlament.al,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,male,589ba883-b893-40ed-95f8-69b10f0497d9,Aldo Bumçi,BUMÇI ALDO,NaN,NaN,PD,c26946f7-8c5e-4474-a85e-ee5342cf4006,tirana_county,Tirana County,7,NaN,NaN,https://www.parlament.al/wp-content/uploads/2015/12/Aldo-Bumci-PD.jpg,Q2832310,Q845743,NaN,490,VII Pluralist Legislature,140,31,292,https://cdn.rawgit.com/everypolitician/everypolitician-data/4ace3b1548b95eb669deab4d99bbc894639269f2/data/Albania/Assembly/term-7.csv,AL,2009-09-08,1557734928,2740502.0,2019,parlament.al
2,bashkim.fino@parlament.al,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,male,9d0181c7-7ebe-4b16-800a-6ea28baf22f5,Bashkim Fino,FINO BASHKIM,NaN,NaN,PS,0c83d4f1-9e86-48f2-95ee-9c3d158d1141,fier_county,Fier County,7,NaN,NaN,https://www.parlament.al/wp-content/uploads/2015/12/Bashkim-Fino-PS.jpg,Q809978,Q642882,NaN,490,VII Pluralist Legislature,140,31,292,https://cdn.rawgit.com/everypolitician/everypolitician-data/4ace3b1548b95eb669deab4d99bbc894639269f2/data/Albania/Assembly/term-7.csv,AL,2009-09-08,1557734928,2740502.0,2019,parlament.al


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8372 entries, 0 to 8371
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   email                   8372 non-null   object
 1   cc3                     8372 non-null   object
 2   country                 8372 non-null   object
 3   ltype                   8372 non-null   object
 4   legislature             8372 non-null   object
 5   chamber                 8372 non-null   object
 6   leg_start_year          8372 non-null   int64 
 7   nemail_cc3              8372 non-null   int64 
 8   gender                  7188 non-null   object
 9   id                      8372 non-null   object
 10  name                    8372 non-null   object
 11  sort_name               8372 non-null   object
 12  twitter                 2375 non-null   object
 13  facebook                1568 non-null   object
 14  group                   8371 non-null   object
 15  grou

In [5]:
# # countries with incomplete gender coverage
# for c in df_ep_emails["cc3"].unique().tolist():
#     _df = df_ep_emails.query(f"cc3=='{c}'")
#     n_gender = len(_df.dropna(subset=["gender"]))
#     n_country = len(_df)
#     if n_gender < n_country:
#         name = _df.reset_index().loc[0, "country"]
#         print(f"{name} ({c}):\n {n_gender}, {n_country}")

In [6]:
set_edomain = set()
for _, row in df_ep_emails.iterrows():
    try:
        edomain = row["email"].split("@")[1]
        set_edomain.add(edomain)
    except IndexError:
        print(_)
#         print(row["email"])

len(set_edomain)

354

In [7]:
# for cc in df_ep_emails["cc3"].unique().tolist():
#     _df = df_ep_emails.query(f"cc3=='{cc}'")
#     _set_edomain = set()
#     for _, row in _df.iterrows():
#         edomain = row["email"].split("@")[1]
#         _set_edomain.add(edomain)

#     print(cc)
#     print(len(_set_edomain))
#     print(_set_edomain)

### Basic numbers

In [8]:
df_ep_emails["nemail_cc3"].describe()

count    8372.000000
mean      283.002867
std       222.489537
min        31.000000
25%       136.000000
50%       208.000000
75%       403.000000
max       832.000000
Name: nemail_cc3, dtype: float64

In [9]:
df_ep_emails["ltype"].unique()

array(['unicameral legislature', 'lower house', 'upper house'],
      dtype=object)

In [10]:
df_ep_emails["legislature"].unique()

array(['Kuvendi', 'Consell General', 'National Assembly',
       'House of Representatives', 'Senate', 'Chamber of Representatives',
       'Parliament', 'House of Commons', 'Assemblée Nationale',
       'Cámara de Representantes', 'Folketing', 'Riigikogu', 'Eduskunta',
       'Parliament of Georgia', 'States', 'Hellenic Parliament',
       'Inatsisartut', 'Congress', 'Legislative Council', 'Országgyűlés',
       'Lok Sabha', 'Majles', 'Chamber of Deputies', 'Parlament',
       'Sobranie', 'National Council', 'Tweede Kamer',
       'Constituent Assembly', 'New Zealand Parliament',
       'National Parliament', 'Assembly', 'House of Assembly',
       'Scottish Parliament', 'National Assembly for Wales'], dtype=object)

In [11]:
df_ep_emails["legislature"].nunique()

34

In [12]:
df_ep_emails["email"].nunique()

8372

In [13]:
df_ep_emails["cc3"].nunique()

55

In [14]:
df_ep_emails.groupby(["cc3", "legislature"]).ngroups

61

## Prep scraped data

In [15]:
df_scraped_emails = (
    pd.read_csv("../data/scraped_pol_combined_legislature_data.csv")
    .sort_values(["cc3", "email"])
    .pipe(lambda df_: clean_dedupe_email_column(df_))
    .drop_duplicates(subset=["email"], keep="first", ignore_index=True)
    # ================================================================
    # Get popsize
    .merge(
        (
            pd.read_csv("../data/popsize.csv")
            .dropna(subset=["cc3"])
            .rename_column("2024 [YR2024]", "pop2024")
        ),
        how="left",
        on="cc3",
        validate="m:1",
    )
    # ================================================================
    # Get #emails per country (cc3)
    .assign(nemail_cc3=lambda df_: df_.groupby("cc3")["email"].transform("nunique"))
    # ================================================================
    .reorder_columns(
        [
            "email",
            "cc3",
            "country",
            "ltype",
            "legislature",
            "chamber",
            #             "year",
            "nemail_cc3",
        ]
    )
    #     # ================================================================
    #     # Renaming fields to harmonise with EP
    .rename_column("leg_start_year", "leg_start_year_sg")
    .assign(
        leg_start_year=lambda df_: np.where(
            df_["cc3"] == "SGP", df_["leg_start_year_sg"], 2025
        ).astype(int)
    )
)
assert (df_scraped_emails["nemail_cc3"] >= 30).all()
display(df_scraped_emails.head(3))
df_scraped_emails.info()

,email,cc3,country,ltype,legislature,chamber,nemail_cc3,name,party,title,address,private_phone,mobile_phone,work_phone,minister_phone,fax,mpsno,initial,firstname,lastname,gender,partyfname,partysname,statename,constname,profession,presentfaddr,presentladdr,delhiphone,permanentfaddr,permanentladdr,personalphone,lastloksabha,lsexpr,age,phone,noofterms,status,imageurl,profileurl,dob,numberofsons,numberofdaughters,qualification,freedom,profession2,categorycode,currentpagenumber,perpagesize,totalelements,totalpages,source_file,mplastfirstname,mpfirstlastname,maritalstatus,createdat,updatedat,email_fix,rank,constituency,leg_start_year_sg,bloque,apellido,nombre,provincia,partido_o_alianza_por_el_que_ingreso,designacion_legal,cese_legal,designacion_real,cese_real,telefono,facebook,twitter,instagram,youtube,nome_parlamentar,partido,uf,titularidade,mandato,telefones,dtnasc,chefe_gab,endereco,state,district,id,parliament_address,parliament_number,social_media,region_x,contact,sr_no,photo,serial_no_,tel_no_r_,constituency_name,permanent_address,mobile,tele_no_res_no_,ac_no,candidate,domain,Country Name,Series Name,Series Code,pop2024,region_y,EU,politician,civil_servants,leg_start_year
0,alejandra.vigo@senado.gob.ar,ARG,Argentina,upper house,Parliament,NaN,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNIDAD FEDERAL,VIGO,ALEJANDRA MARÍA,CÓRDOBA,HACEMOS POR CÓRDOBA,2021-12-10,2027-12-09,2021-12-10,Sin Datos,1441 / 1444 / 1456,https://www.facebook.com/VigoAlejandra,https://twitter.com/@alevigo,https://www.instagram.com/@vigo_alejandra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,senado.gob.ar,Argentina,"Population, total",SP.POP.TOTL,46936024,NaN,NaN,NaN,NaN,2025
1,alfredo.deangeli@senado.gob.ar,ARG,Argentina,upper house,Parliament,NaN,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRENTE PRO,DE ANGELI,ALFREDO LUIS,ENTRE RÍOS,JUNTOS POR EL CAMBIO,2019-12-10,2025-12-09,2019-12-10,Sin Datos,3580 / 81 / 82 / 84,NaN,https://twitter.com/alfredodeangeli,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,senado.gob.ar,Argentina,"Population, total",SP.POP.TOTL,46936024,NaN,NaN,NaN,NaN,2025
2,alicia.kirchner@senado.gob.ar,ARG,Argentina,upper house,Parliament,NaN,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNIDAD CIUDADANA,KIRCHNER,ALICIA MARGARITA ANTONIA,SANTA CRUZ,ALIANZA UNIÓN POR LA PATRIA,2023-12-10,2029-12-09,2023-12-10,Sin Datos,1389 / 1390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,senado.gob.ar,Argentina,"Population, total",SP.POP.TOTL,46936024,NaN,NaN,NaN,NaN,2025


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4551 entries, 0 to 4550
Columns: 112 entries, email to leg_start_year
dtypes: float64(21), int64(2), object(89)
memory usage: 3.9+ MB


In [16]:
df_scraped_emails.groupby("cc3").size()

cc3
ARG      71
BRA      81
DNK     186
GRC     368
IND    3233
NGA      68
NOR     174
SGP     370
dtype: int64

### Basic numbers

In [17]:
df_scraped_emails["nemail_cc3"].describe()

count    4551.000000
mean     2374.359701
std      1346.464748
min        68.000000
25%       370.000000
50%      3233.000000
75%      3233.000000
max      3233.000000
Name: nemail_cc3, dtype: float64

In [18]:
df_scraped_emails["legislature"].unique()

array(['Parliament', 'Folketing', 'Lok Sabha', 'State Legislature',
       'Senate', 'Storting'], dtype=object)

In [19]:
df_scraped_emails["email"].nunique()

4551

In [20]:
df_scraped_emails["name"].nunique()

1494

In [21]:
df_scraped_emails["cc3"].nunique()

8

In [22]:
df_scraped_emails.groupby(["cc3", "legislature"]).ngroups

9

In [23]:
set_edomain = set()
for _, row in df_scraped_emails.iterrows():
    try:
        edomain = row["email"].split("@")[1]
        set_edomain.add(edomain)
    except IndexError:
        print(_)
#         print(row["email"])

len(set_edomain)

233

In [24]:
# for cc in df_scraped_emails["cc3"].unique().tolist():
#     _df = df_scraped_emails.query(f"cc3=='{cc}'")
#     _set_edomain = set()
#     for _, row in _df.iterrows():
#         edomain = row["email"].split("@")[1]
#         _set_edomain.add(edomain)

#     print(cc)
#     print(len(_set_edomain))
#     print(_set_edomain)

## Combine EP + Scraped

In [25]:
df_pol_emails = (
    pd.concat(
        [df_ep_emails.assign(source="ep"), df_scraped_emails.assign(source="scraped")],
        ignore_index=True,
    )
    .pipe(lambda df_: clean_dedupe_email_column(df_))
    .drop_duplicates(subset=["email"], keep="first", ignore_index=True)
    # ================================================================
    # Get indicator for years and chambers for table1
    .assign(
        years=lambda df_: df_.groupby("cc3")["leg_start_year"].transform(
            lambda x: ", ".join(map(str, sorted(x.unique())))
        ),
        chambers=lambda df_: df_.groupby("cc3")["ltype"]
        .transform(lambda x: ", ".join(map(str, sorted(x.unique()))))
        .replace("legislature", "", regex=True)
        .replace("house", "", regex=True)
        .str.strip()
        .str.title(),
        legislatures=lambda df_: df_.groupby("cc3")["legislature"].transform(
            lambda x: ", ".join(map(str, sorted(x.unique())))
        ),
    )
    # ================================================================
    # Get #emails per country (cc3)
    .assign(nemail_cc3=lambda df_: df_.groupby("cc3")["email"].transform("nunique"))
    .sort_values(["cc3", "leg_start_year", "email"], ignore_index=True)
)
# del df_ep_emails, df_scraped_emails

assert (df_pol_emails["nemail_cc3"] >= 30).all()
display(df_pol_emails.head(3))
df_pol_emails.info()

,email,cc3,country,ltype,legislature,chamber,leg_start_year,nemail_cc3,gender,id,name,sort_name,twitter,facebook,group,group_id,area_id,area,term_x,start_date,end_date,image,wikidata,wikidata_group,wikidata_area,row_id,term_y,row_count,n_unique_emails,person_count_legistype,url,cc,leg_start_date,lastmod,pop2024,lastmod_year,domain,source,party,title,address,private_phone,mobile_phone,work_phone,minister_phone,fax,mpsno,initial,firstname,lastname,partyfname,partysname,statename,constname,profession,presentfaddr,presentladdr,delhiphone,permanentfaddr,permanentladdr,personalphone,lastloksabha,lsexpr,age,phone,noofterms,status,imageurl,profileurl,dob,numberofsons,numberofdaughters,qualification,freedom,profession2,categorycode,currentpagenumber,perpagesize,totalelements,totalpages,source_file,mplastfirstname,mpfirstlastname,maritalstatus,createdat,updatedat,email_fix,rank,constituency,leg_start_year_sg,bloque,apellido,nombre,provincia,partido_o_alianza_por_el_que_ingreso,designacion_legal,cese_legal,designacion_real,cese_real,telefono,instagram,youtube,nome_parlamentar,partido,uf,titularidade,mandato,telefones,dtnasc,chefe_gab,endereco,state,district,parliament_address,parliament_number,social_media,region_x,contact,sr_no,photo,serial_no_,tel_no_r_,constituency_name,permanent_address,mobile,tele_no_res_no_,ac_no,candidate,Country Name,Series Name,Series Code,region_y,EU,politician,civil_servants,years,chambers,legislatures
0,albana.vokshi@parlament.al,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,female,c8bd71fc-4815-40b5-a5c4-bd359e3b0cef,Albana Vokshi,VOKSHI ALBANA,NaN,NaN,PD,c26946f7-8c5e-4474-a85e-ee5342cf4006,durrës_county,Durrës County,7,NaN,NaN,https://www.parlament.al/wp-content/uploads/2015/12/Albana-Vokshi-PD.jpg,Q4709025,Q845743,NaN,490.0,VII Pluralist Legislature,140.0,31.0,292.0,https://cdn.rawgit.com/everypolitician/everypolitician-data/4ace3b1548b95eb669deab4d99bbc894639269f2/data/Albania/Assembly/term-7.csv,AL,2009-09-08,1.557735e+09,2740502.0,2019.0,parlament.al,ep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2009, 2013, 2017",Unicameral,Kuvendi
1,aldo.bumci@parlament.al,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,male,589ba883-b893-40ed-95f8-69b10f0497d9,Aldo Bumçi,BUMÇI ALDO,NaN,NaN,PD,c26946f7-8c5e-4474-a85e-ee5342cf4006,tirana_county,Tirana County,7,NaN,NaN,https://www.parlament.al/wp-content/uploads/2015/12/Aldo-Bumci-PD.jpg,Q2832310,Q845743,NaN,490.0,VII Pluralist Legislature,140.0,31.0,292.0,https://cdn.rawgit.com/everypolitician/everypolitician-data/4ace3b1548b95eb669deab4d99bbc894639269f2/data/Albania/Assembly/term-7.csv,AL,2009-09-08,1.557735e+09,2740502.0,2019.0,parlament.al,ep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2009, 2013, 2017",Unicameral,Kuvendi
2,bashkim.fino@parlament.al,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,male,9d0181c7-7ebe-4b16-800a-6ea28baf22f5,Bashkim Fino,FINO BASHKIM,NaN,NaN,PS,0c83d4f1-9e86-48f2-95ee-9c3d158d1141,fier_county,Fier County,7,NaN,NaN,https://www.parlament.al/wp-content/uploads/2015/12/Bashkim-Fino-PS.jpg,Q809978,Q642882,NaN,490.0,VII Pluralist Legislature,140.0,31.0,292.0,https://cdn.rawgit.com/everypolitician/everypolitician-data/4ace3b1548b95eb669deab4d99bbc894639269f2/data/Albania/Assembly/term-7.csv,AL,2009-09-08,1.557735e+09,2740502.0,2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12391 entries, 0 to 12390
Columns: 138 entries, email to legislatures
dtypes: float64(26), int64(2), object(110)
memory usage: 13.0+ MB


In [26]:
df_pol_emails["nemail_cc3"].describe()

count    12391.000000
mean      1108.329029
std       1363.308653
min         31.000000
25%        158.000000
50%        370.000000
75%       3334.000000
max       3334.000000
Name: nemail_cc3, dtype: float64

In [27]:
df_pol_emails["email"].nunique()

12391

In [28]:
df_pol_emails["cc3"].nunique()

59

In [29]:
df_pol_emails.groupby(["cc3", "legislature"]).ngroups

67

In [30]:
set_edomains = set()
for cc in df_pol_emails["cc3"].unique().tolist():
    _df = df_pol_emails.query(f"cc3=='{cc}'")
    _set_edomain = set()
    for _, row in _df.iterrows():
        edomain = row["email"].split("@")[1]
        _set_edomain.add(edomain)
        set_edomains.add(edomain)

#     print(cc)
#     print(len(_set_edomain))
#     print(_set_edomain)

In [31]:
len(set_edomains)

531

## Table1

In [32]:
df_tab1 = (
    df_pol_emails.drop_duplicates(["cc3"], ignore_index=True)
    #     .filter(["cc3", "country", "nemail_cc3", "female_prop_cc3", "years", "chambers", "legislatures", "pop2024"])
    #     .assign(female_prop_cc3=lambda df_: (100 * df_["female_prop_cc3"]).round(1).astype(str) + "\\%")
    .filter(
        ["cc3", "country", "nemail_cc3", "years", "chambers", "legislatures", "pop2024"]
    )
    .assign(
        pop2024=lambda df_: (df_["pop2024"].astype(float) / 1_000_000)
        .round(1)
        .apply(str)
    )
    .replace("Bosnia-and-Herzegovina", "Bosnia")
    .replace(
        "House of Commons, National Assembly for Wales, Scottish Parliament",
        "Commons, Senedd, Scottish Parliament",
    )
    # Tidy year strings
    .replace("1997, 2001, 2005, 2007, 2010, 2011, 2015, 2016, 2017", "1997--2017")
    .replace("2004, 2007, 2010, 2013, 2016", "2004--2016")
    .replace("2001, 2005, 2007, 2011, 2015", "2001--2015")
    .replace("2004, 2007, 2009, 2012, 2015", "2004--2015")
    .replace("2001, 2005, 2007, 2011, 2015, 2025", "2001--2025")
    .replace("2008, 2011, 2014, 2017", "2008--2017")
    .replace("2006, 2010, 2012, 2016", "2006--2016")
    .replace("2001, 2006, 2011, 2015, 2021", "2001--2025")
    .replace("2004, 2007, 2009, 2012, 2015, 2025", "2004--2025")
    .replace("Lower ", "Lower", regex=True)
    .replace("Upper, Lower, Upper", "Lower, Upper", regex=True)
    .assign(ix=lambda df_: range(1, 1 + len(df_)))
    .reorder_columns(["ix"])
)
df_tab1

,ix,cc3,country,nemail_cc3,years,chambers,legislatures,pop2024
0,1,ALB,Albania,140,"2009, 2013, 2017",Unicameral,Kuvendi,2.7
1,2,AND,Andorra,31,2015,Unicameral,Consell General,0.1
2,3,ARG,Argentina,71,2025,Upper,Parliament,46.9
3,4,ARM,Armenia,119,2019,Unicameral,National Assembly,2.8
4,5,AUS,Australia,177,2004--2016,"Lower, Upper","House of Representatives, Senate",26.9
5,6,BEL,Belgium,149,2014,Lower,Chamber of Representatives,11.9
6,7,BGR,Bulgaria,205,"2013, 2014, 2017",Unicameral,National Assembly,6.4
7,8,BIH,Bosnia,42,2014,Lower,House of Representatives,3.2
8,9,BLR,Belarus,59,2016,Unicameral,House of Representatives,9.1
9,10,BMU,Bermuda,33,2017,Lower,Parliament,nan


In [33]:
df_tab1["nemail_cc3"].sum()

np.int64(12391)

In [34]:
df_tab1["pop2024"].astype(float).sum()

np.float64(2884.5)

In [35]:
# % coverage of global pop
100 * (df_tab1["pop2024"].astype(float).sum()) / 8000

np.float64(36.05625)

In [36]:
pandas_to_tex(
    df_tab1.replace("nan", "---"),
    "../tables/hipb_pooled_emailcoverage_summary.tex",
    index=False,
)

In [37]:
!cat ../tables/hipb_pooled_emailcoverage_summary.tex

\midrule
1 & ALB & Albania & 140 & 2009, 2013, 2017 & Unicameral & Kuvendi & 2.7 \\
2 & AND & Andorra & 31 & 2015 & Unicameral & Consell General & 0.1 \\
3 & ARG & Argentina & 71 & 2025 & Upper & Parliament & 46.9 \\
4 & ARM & Armenia & 119 & 2019 & Unicameral & National Assembly & 2.8 \\
5 & AUS & Australia & 177 & 2004--2016 & Lower, Upper & House of Representatives, Senate & 26.9 \\
6 & BEL & Belgium & 149 & 2014 & Lower & Chamber of Representatives & 11.9 \\
7 & BGR & Bulgaria & 205 & 2013, 2014, 2017 & Unicameral & National Assembly & 6.4 \\
8 & BIH & Bosnia & 42 & 2014 & Lower & House of Representatives & 3.2 \\
9 & BLR & Belarus & 59 & 2016 & Unicameral & House of Representatives & 9.1 \\
10 & BMU & Bermuda & 33 & 2017 & Lower & Parliament & --- \\
11 & BRA & Brazil & 81 & 2025 & Upper & Parliament & 217.6 \\
12 & BTN & Bhutan & 46 & 2013 & Lower & National Assembly & 0.8 \\
13 & CAN & Canada & 432 & 2011, 2015 & Lower, Upper & House of Commons, Senate & 40.4 \\
14 & CMR & Camer

## Merge to HIBP

### EP data

In [38]:
df_ep_email_breach_expanded = (
    # All EP emails
    df_ep_emails.filter(
        [
            "email",
            "gender",
            "cc3",
            "country",
            "ltype",
            "legislature",
            "chamber",
            "leg_start_year",
            "nemail_cc3",
        ]
    )
    # ========================================================================
    # Merge to breached status
    .merge(
        (
            pd.read_csv("../data/everypol_hibp.csv")
            .clean_names()
            .rename_column("filename", "email")
            # needed to normalize strings (e.g., unicode/ASCII) -- do not remove
            .pipe(lambda df_: clean_dedupe_email_column(df_, dedup=False))
        ),
        how="left",
        on="email",
        validate="1:m",
    ).dropna(subset=["breach"])
    # ========================================================================
    # Merge to expanded breach info
    .merge(
        (
            pd.read_parquet("../data/breaches_01_2025_expanded.parquet")
            .remove_columns(
                [
                    "description",
                    "title",
                    "domain",
                    "breachdate",
                    "addeddate",
                    "modifieddate",
                    "logopath",
                    "timetopublic",
                    "dataclasses",
                ]
            )
            .rename_column("name", "breach")
        ),
        how="left",
        on="breach",
        validate="m:1",
    )
    # ========================================================================
    #     .query("present==True")
    .reset_index(drop=True)
)
df_ep_email_breach_expanded.head(3)

,email,gender,cc3,country,ltype,legislature,chamber,leg_start_year,nemail_cc3,breach,present,domain,pwncount,isverified,isfabricated,issensitive,isretired,isspamlist,ismalware,issubscriptionfree,yearstopublic,n_dataclasses,Education levels,Private messages,Personal health data,Licence plates,Apps installed on devices,Financial transactions,Family structure,Support tickets,Eating habits,Religions,Vehicle identification numbers (VINs),Sexual orientations,Instant messenger identities,Deceased date,Website activity,Security questions and answers,Passwords,Account balances,Customer feedback,Home ownership statuses,Net worths,Tattoo status,Loan information,User statuses,Health insurance information,Car ownership statuses,Telecommunications carrier,Income levels,Career levels,Mothers maiden names,Bank account numbers,Password strengths,Work habits,Historical passwords,Physical attributes,Parenting plans,Time zones,Email messages,Living costs,Sexual fetishes,Family members' names,Geographic locations,Political views,Government issued IDs,Personal interests,MAC addresses,Browsing histories,Device information,Places of birth,Partial credit card data,Years of professional experience,Dates of birth,Utility bills,...,HIV statuses,Company names,Beauty ratings,Political donations,Races,Survey results,Flights taken,Credit card CVV,Drug habits,Usernames,Spoken languages,Social security numbers,Ages,Chat logs,Comments,Cellular network names,Social media profiles,Browser user agent details,Citizenship statuses,Travel plans,Purchasing habits,Device usage tracking data,Credit status information,Spouses names,Relationship statuses,Auth tokens,Charitable donations,Nationalities,Biometric data,Recovery email addresses,Taxation records,Encrypted keys,Delivery instructions,Payment methods,Salutations,Fitness levels,Email addresses,Audio recordings,Job titles,User website URLs,Reward program balances,Age groups,Physical addresses,Employment statuses,Login histories,Driver's licenses,IMSI numbers,Names,Avatars,Employers,Appointments,Travel habits,Purchases,Customer interactions,Buying preferences,Warranty claims,Partial dates of birth,Ethnicities,Mnemonic phrases,Marital statuses,Payment histories,Social connections,IMEI numbers,Password hints,Vehicle details,IP addresses,Financial investments,PINs,Occupations,Cryptocurrency wallet addresses,SMS messages,Astrological signs,Genders,Credit cards,Personal descriptions
0,albana.vokshi@parlament.al,female,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,000webhost,False,parlament.al,14936670,True,False,False,False,False,False,False,0.654795,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,albana.vokshi@parlament.al,female,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,123RF,False,parlament.al,8661578,True,False,False,False,False,False,False,0.652055,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,albana.vokshi@parlament.al,female,ALB,Albania,unicameral legislature,Kuvendi,Kuvendi,2009,140,2844Breaches,False,parlament.al,80115532,False,False,False,False,False,False,False,0.019178,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Scraped data

In [40]:
df_scraped_email_breach_expanded = (
    df_scraped_emails.filter(["email", "cc3", "country", "year", "nemail_cc3"])
    # ============================================================================
    # Merge to breached status
    .merge(
        (
            pd.read_csv("../data/scraped_pol_hibp.csv")
            .clean_names()
            .rename_column("filename", "email")
        ),
        how="left",
        on="email",
        validate="1:m",
    )
    #     .dropna(subset=["breach"])
    .assign(present=lambda df_: df_["present"].fillna(False))
    # ============================================================================
    # Merge to expanded breach info
    .merge(
        (
            pd.read_parquet("../data/breaches_01_2025_expanded.parquet")
            .remove_columns(
                [
                    "description",
                    "title",
                    "domain",
                    "breachdate",
                    "addeddate",
                    "modifieddate",
                    "logopath",
                    "timetopublic",
                    "dataclasses",
                ]
            )
            .rename_column("name", "breach")
        ),
        how="left",
        on="breach",
        validate="m:1",
    )
    # ============================================================================
    #     .query("present==True")
    .reset_index(drop=True)
)
df_scraped_email_breach_expanded.head()

,email,cc3,country,nemail_cc3,breach,present,pwncount,isverified,isfabricated,issensitive,isretired,isspamlist,ismalware,issubscriptionfree,yearstopublic,n_dataclasses,Education levels,Private messages,Personal health data,Licence plates,Apps installed on devices,Financial transactions,Family structure,Support tickets,Eating habits,Religions,Vehicle identification numbers (VINs),Sexual orientations,Instant messenger identities,Deceased date,Website activity,Security questions and answers,Passwords,Account balances,Customer feedback,Home ownership statuses,Net worths,Tattoo status,Loan information,User statuses,Health insurance information,Car ownership statuses,Telecommunications carrier,Income levels,Career levels,Mothers maiden names,Bank account numbers,Password strengths,Work habits,Historical passwords,Physical attributes,Parenting plans,Time zones,Email messages,Living costs,Sexual fetishes,Family members' names,Geographic locations,Political views,Government issued IDs,Personal interests,MAC addresses,Browsing histories,Device information,Places of birth,Partial credit card data,Years of professional experience,Dates of birth,Utility bills,Deceased statuses,Job applications,Drinking habits,Nicknames,Passport numbers,Smoking habits,...,HIV statuses,Company names,Beauty ratings,Political donations,Races,Survey results,Flights taken,Credit card CVV,Drug habits,Usernames,Spoken languages,Social security numbers,Ages,Chat logs,Comments,Cellular network names,Social media profiles,Browser user agent details,Citizenship statuses,Travel plans,Purchasing habits,Device usage tracking data,Credit status information,Spouses names,Relationship statuses,Auth tokens,Charitable donations,Nationalities,Biometric data,Recovery email addresses,Taxation records,Encrypted keys,Delivery instructions,Payment methods,Salutations,Fitness levels,Email addresses,Audio recordings,Job titles,User website URLs,Reward program balances,Age groups,Physical addresses,Employment statuses,Login histories,Driver's licenses,IMSI numbers,Names,Avatars,Employers,Appointments,Travel habits,Purchases,Customer interactions,Buying preferences,Warranty claims,Partial dates of birth,Ethnicities,Mnemonic phrases,Marital statuses,Payment histories,Social connections,IMEI numbers,Password hints,Vehicle details,IP addresses,Financial investments,PINs,Occupations,Cryptocurrency wallet addresses,SMS messages,Astrological signs,Genders,Credit cards,Personal descriptions
0,alejandra.vigo@senado.gob.ar,ARG,Argentina,71,StealerLogsJan2025,False,71039833.0,True,False,False,False,False,False,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,alfredo.deangeli@senado.gob.ar,ARG,Argentina,71,StealerLogsJan2025,False,71039833.0,True,False,False,False,False,False,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,alicia.kirchner@senado.gob.ar,ARG,Argentina,71,StealerLogsJan2025,False,71039833.0,True,False,False,False,False,False,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

### Manually check

In [41]:
# non_breached_emails = []
# for pol in df["email"].unique():
#     if pol not in df_ep_email_breach_expanded["email"].unique().tolist():
#         non_breached_emails.append(pol)

In [42]:
# import random
# random.seed(42)

# # Sample 10 unique emails
# sampled_emails = random.sample(non_breached_emails, 20)
# sampled_emails

Paste (no breach) 

* g.akriotis@parliament.gr

### Combine

In [43]:
df_email_breach_expanded = (
    pd.concat(
        [
            df_ep_email_breach_expanded.remove_columns(
                ["gender", "country", "ltype", "chamber", "legislature"]
            ).rename_column("leg_start_year", "year"),
            df_scraped_email_breach_expanded.remove_columns(["country"]),
        ],
        ignore_index=True,
    )
    .dropna(subset=["breach"])
    .assign(present=lambda df_: df_["present"].astype(bool))
    #     # ========================================================================
    .assign(
        seriousbreach=lambda df_: df_[LIST_SERIOUS_DATACLASSES].max(axis=1).astype(int)
    )
    # ========================================================================
    .pipe(lambda df_: classify_comm_gov_email(df_))
)
df_email_breach_expanded

,email,cc3,year,nemail_cc3,breach,present,domain,pwncount,isverified,isfabricated,issensitive,isretired,isspamlist,ismalware,issubscriptionfree,yearstopublic,n_dataclasses,Education levels,Private messages,Personal health data,Licence plates,Apps installed on devices,Financial transactions,Family structure,Support tickets,Eating habits,Religions,Vehicle identification numbers (VINs),Sexual orientations,Instant messenger identities,Deceased date,Website activity,Security questions and answers,Passwords,Account balances,Customer feedback,Home ownership statuses,Net worths,Tattoo status,Loan information,User statuses,Health insurance information,Car ownership statuses,Telecommunications carrier,Income levels,Career levels,Mothers maiden names,Bank account numbers,Password strengths,Work habits,Historical passwords,Physical attributes,Parenting plans,Time zones,Email messages,Living costs,Sexual fetishes,Family members' names,Geographic locations,Political views,Government issued IDs,Personal interests,MAC addresses,Browsing histories,Device information,Places of birth,Partial credit card data,Years of professional experience,Dates of birth,Utility bills,Deceased statuses,Job applications,Drinking habits,Nicknames,Passport numbers,...,Beauty ratings,Political donations,Races,Survey results,Flights taken,Credit card CVV,Drug habits,Usernames,Spoken languages,Social security numbers,Ages,Chat logs,Comments,Cellular network names,Social media profiles,Browser user agent details,Citizenship statuses,Travel plans,Purchasing habits,Device usage tracking data,Credit status information,Spouses names,Relationship statuses,Auth tokens,Charitable donations,Nationalities,Biometric data,Recovery email addresses,Taxation records,Encrypted keys,Delivery instructions,Payment methods,Salutations,Fitness levels,Email addresses,Audio recordings,Job titles,User website URLs,Reward program balances,Age groups,Physical addresses,Employment statuses,Login histories,Driver's licenses,IMSI numbers,Names,Avatars,Employers,Appointments,Travel habits,Purchases,Customer interactions,Buying preferences,Warranty claims,Partial dates of birth,Ethnicities,Mnemonic phrases,Marital statuses,Payment histories,Social connections,IMEI numbers,Password hints,Vehicle details,IP addresses,Financial investments,PINs,Occupations,Cryptocurrency wallet addresses,SMS messages,Astrological signs,Genders,Credit cards,Personal descriptions,seriousbreach,ecategory
0,albana.vokshi@parlament.al,ALB,2009.0,140,000webhost,False,parlament.al,14936670.0,True,False,False,False,False,False,False,0.654795,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Official
1,albana.vokshi@parlament.al,ALB,2009.0,140,123RF,False,parlament.al,8661578.0,True,False,False,False,False,False,False,0.652055,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Official
2,albana.vokshi@parlament.al,ALB,2009.0,140,2844Breaches,False,parlament.al,80115532.0,False,False,False,False,False,False,False,0.019178,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [44]:
# (
#     df_email_breach_expanded
#     .query("ecategory=='Official'")
#     .drop_duplicates(["email"])
#     .groupby("domain")
#     .size()
#     .reset_index(name="count")
#     .sort_values("count", ascending=False, ignore_index=True)
#     .head(100)
# )

In [45]:
# (
#     df_email_breach_expanded
#     .query("ecategory=='Commercial'")
#     .drop_duplicates(["email"])
#     .groupby("domain")
#     .size()
#     .reset_index(name="count")
#     .sort_values("count", ascending=False, ignore_index=True)
#     .head(50)
# )

### Basic numbers

In [46]:
n_emails = df_email_breach_expanded["email"].nunique()
n_emails

12384

In [47]:
# pols involved in 562 data breaches
df_email_breach_expanded["breach"].nunique()

562

In [48]:
# 3750 pols have been breached
n_pwned_pols = df_email_breach_expanded.query("present==True")["email"].nunique()
n_pwned_pols

4092

In [49]:
100 * n_pwned_pols / df_email_breach_expanded["email"].nunique()

33.042635658914726

In [50]:
# 2620 pols have been breached multiple times
n_multi_pwned_pols = (
    df_email_breach_expanded.groupby(["email"])["present"]
    .sum()
    .reset_index()
    .query("present>1")["email"]
    .nunique()
)
n_multi_pwned_pols

2620

In [51]:
# Proportion of multiple breaches
100 * (n_multi_pwned_pols / n_pwned_pols)

64.02737047898339

In [52]:
# Proportion of total
100 * n_multi_pwned_pols / df_email_breach_expanded["email"].nunique()

21.156330749354005

In [53]:
# How many email-pw pairs
_n = (
    df_email_breach_expanded.query("present==True")
    .clean_names()
    .query("email_addresses==1")
    .query("passwords==1")["email"]
    .nunique()
)

In [54]:
100 * _n / n_pwned_pols

63.44086021505376

In [55]:
# How many had serious breaches
n_pwned_pols_sbreach = (
    df_email_breach_expanded.query("present==True")
    .query("seriousbreach==1")["email"]
    .nunique()
)
n_pwned_pols_sbreach

2672

In [56]:
# How many had multiple serious breaches
n_multi_pwned_pols_sbreach = (
    df_email_breach_expanded.query("present==True")
    .query("seriousbreach==1")
    .groupby(["email"])["seriousbreach"]
    .sum()
    .reset_index()
    .query("seriousbreach>1")["email"]
    .nunique()
)
n_multi_pwned_pols_sbreach

1401

### Basic numbers - Personal/Comm

In [57]:
n_comm_emails = df_email_breach_expanded.query("ecategory=='Commercial'")[
    "email"
].nunique()
n_comm_emails

2766

In [58]:
# pols involved in 562 data breaches
df_email_breach_expanded.query("ecategory=='Commercial'")["breach"].nunique()

562

In [59]:
# 1301 pols have been breached
n_pwned_pols_comm = (
    df_email_breach_expanded.query("ecategory=='Commercial'")
    .query("present==True")["email"]
    .nunique()
)
n_pwned_pols_comm

1301

In [60]:
100 * n_pwned_pols_comm / df_email_breach_expanded.query("ecategory=='Commercial'")[
    "email"
].nunique()

47.035430224150396

In [61]:
# 867 pols have been breached multiple times
n_multi_pwned_pols_comm = (
    df_email_breach_expanded.query("ecategory=='Commercial'")
    .groupby(["email"])["present"]
    .sum()
    .reset_index()
    .query("present>1")["email"]
    .nunique()
)
n_multi_pwned_pols_comm

867

In [62]:
# Proportion of multiple breaches
100 * (n_multi_pwned_pols_comm / n_pwned_pols_comm)

66.64104534973097

In [63]:
# How many email-pw pairs
_n = (
    df_email_breach_expanded.query("ecategory=='Commercial'")
    .query("present==True")
    .clean_names()
    .query("email_addresses==1")
    .query("passwords==1")["email"]
    .nunique()
)

In [64]:
# How many had serious breaches
n_pwned_pols_sbreach_comm = (
    df_email_breach_expanded.query("ecategory=='Commercial'")
    .query("present==True")
    .query("seriousbreach==1")["email"]
    .nunique()
)
n_pwned_pols_sbreach_comm

964

In [65]:
# How many had multiple serious breaches
n_multi_pwned_pols_sbreach_comm = (
    df_email_breach_expanded.query("ecategory=='Commercial'")
    .query("present==True")
    .query("seriousbreach==1")
    .groupby("email")["seriousbreach"]
    .sum()
    .reset_index()
    .query("seriousbreach>1")["email"]
    .nunique()
)
n_multi_pwned_pols_sbreach_comm

584

In [66]:
# How many had multiple serious breaches
n_multi_pwned_pols_sbreach = (
    df_email_breach_expanded.query("present==True")
    .query("seriousbreach==1")
    .groupby("email")["seriousbreach"]
    .sum()
    .reset_index()
    .query("seriousbreach>1")["email"]
    .nunique()
)
n_multi_pwned_pols_sbreach

1401

### Basic numbers - Official

In [67]:
n_govt_emails = df_email_breach_expanded.query("ecategory=='Official'")[
    "email"
].nunique()
n_govt_emails

9618

In [68]:
# pols involved in 563 data breaches
df_email_breach_expanded.query("ecategory=='Official'")["breach"].nunique()

561

In [69]:
# 2791 pols have been breached
n_pwned_pols_govt = (
    df_email_breach_expanded.query("ecategory=='Official'")
    .query("present==True")["email"]
    .nunique()
)
n_pwned_pols_govt

2791

In [70]:
100 * n_pwned_pols_govt / n_govt_emails

29.01850696610522

In [71]:
# 867 pols have been breached multiple times
n_multi_pwned_pols_govt = (
    df_email_breach_expanded.query("ecategory=='Official'")
    .groupby(["email"])["present"]
    .sum()
    .reset_index()
    .query("present>1")["email"]
    .nunique()
)
n_multi_pwned_pols_comm

867

In [72]:
# Proportion of multiple breaches
100 * (n_multi_pwned_pols_govt / n_pwned_pols_govt)

62.80902902185596

In [73]:
# How many email-pw pairs
_n = (
    df_email_breach_expanded.query("ecategory=='Official'")
    .query("present==True")
    .clean_names()
    .query("email_addresses==1")
    .query("passwords==1")["email"]
    .nunique()
)

In [74]:
100 * _n / n_pwned_pols_govt

59.26191329272662

In [75]:
# How many had serious breaches
n_pwned_pols_sbreach_govt = (
    df_email_breach_expanded.query("ecategory=='Official'")
    .query("present==True")
    .query("seriousbreach==1")["email"]
    .nunique()
)
n_pwned_pols_sbreach_govt

1708

In [76]:
# How many had multiple serious breaches
n_multi_pwned_pols_sbreach_govt = (
    df_email_breach_expanded.query("ecategory=='Official'")
    .query("present==True")
    .query("seriousbreach==1")
    .groupby(["email"])["seriousbreach"]
    .sum()
    .reset_index()
    .query("seriousbreach>1")["email"]
    .nunique()
)
n_multi_pwned_pols_sbreach_govt

817

## Tabulate

### Breach summary

In [77]:
_all = (
    df_email_breach_expanded.groupby("email")["present"]
    .sum()
    .pipe(lambda s: s.describe().to_frame().T)
    .assign(perc_at_least_1=100 * n_pwned_pols / n_emails)
    .assign(perc_at_least_2=100 * n_multi_pwned_pols / n_emails)
)
_all

,count,mean,std,min,25%,50%,75%,max,perc_at_least_1,perc_at_least_2
present,12384.0,1.072432,3.280699,0.0,0.0,0.0,1.0,256.0,33.042636,21.156331


In [78]:
_govt = (
    df_email_breach_expanded.query("ecategory=='Official'")
    .groupby("email")["present"]
    .sum()
    .pipe(lambda s: s.describe().to_frame().T)
    .astype(int)
    .assign(perc_at_least_1=100 * n_pwned_pols_govt / n_govt_emails)
    .assign(perc_at_least_2=100 * n_multi_pwned_pols_govt / n_govt_emails)
)
_govt

,count,mean,std,min,25%,50%,75%,max,perc_at_least_1,perc_at_least_2
present,9618,0,1,0,0,0,1,36,29.018507,18.226242


In [79]:
_comm = (
    df_email_breach_expanded.query("ecategory=='Commercial'")
    .groupby("email")["present"]
    .sum()
    .pipe(lambda s: s.describe().to_frame().T)
    .astype(int)
    .assign(perc_at_least_1=100 * n_pwned_pols_comm / n_comm_emails)
    .assign(perc_at_least_2=100 * n_multi_pwned_pols_comm / n_comm_emails)
)
_comm

,count,mean,std,min,25%,50%,75%,max,perc_at_least_1,perc_at_least_2
present,2766,1,5,0,0,0,2,256,47.03543,31.344902


In [80]:
_tab = (
    pd.concat([_all, _govt, _comm], ignore_index=True)
    .assign(
        **{
            col: lambda df_, col=col: df_[col].round(0).astype(int)
            for col in ["min", "25%", "50%", "75%", "max"]
        }
    )
    .round(1)
    .assign(perc_at_least_1=lambda df_: df_["perc_at_least_1"].astype(str) + "\\%")
    .assign(perc_at_least_2=lambda df_: df_["perc_at_least_2"].astype(str) + "\\%")
    .assign(
        count=lambda df_: df_["count"].round(0).astype(int).apply(lambda x: f"{x:,}")
    )
    .astype(str)
    .assign(cat=["All emails", "Government emails", "Personal emails"])
    .reorder_columns(["cat"])
)
pandas_to_tex(_tab, "../tables/pooled_pols_breach_number_summary", index=False)
display(_tab)
!cat ../tables/pooled_pols_breach_number_summary.tex

,cat,count,mean,std,min,25%,50%,75%,max,perc_at_least_1,perc_at_least_2
0,All emails,"12,384",1.1,3.3,0,0,0,1,256,33.0\%,21.2\%
1,Government emails,"9,618",0.0,1.0,0,0,0,1,36,29.0\%,18.2\%
2,Personal emails,"2,766",1.0,5.0,0,0,0,2,256,47.0\%,31.3\%


\midrule
All emails & 12,384 & 1.1 & 3.3 & 0 & 0 & 0 & 1 & 256 & 33.0\% & 21.2\% \\
Government emails & 9,618 & 0.0 & 1.0 & 0 & 0 & 0 & 1 & 36 & 29.0\% & 18.2\% \\
Personal emails & 2,766 & 1.0 & 5.0 & 0 & 0 & 0 & 2 & 256 & 47.0\% & 31.3\% \\

In [81]:
_all = (
    df_email_breach_expanded.query("seriousbreach==1")
    .groupby("email")["present"]
    .sum()
    .pipe(lambda s: s.describe().to_frame().T)
    .assign(perc_at_least_1=100 * n_pwned_pols_sbreach / n_emails)
    .assign(perc_at_least_2=100 * n_multi_pwned_pols_sbreach / n_emails)
)
_all

,count,mean,std,min,25%,50%,75%,max,perc_at_least_1,perc_at_least_2
present,12384.0,0.508317,2.17221,0.0,0.0,0.0,0.0,189.0,21.576227,11.312984


In [82]:
_govt = (
    df_email_breach_expanded.query("ecategory=='Official'")
    .query("seriousbreach==1")
    .groupby("email")["present"]
    .sum()
    .pipe(lambda s: s.describe().to_frame().T)
    .astype(int)
    .assign(perc_at_least_1=100 * n_pwned_pols_sbreach_govt / n_govt_emails)
    .assign(perc_at_least_2=100 * n_multi_pwned_pols_sbreach_govt / n_govt_emails)
)
_govt

,count,mean,std,min,25%,50%,75%,max,perc_at_least_1,perc_at_least_2
present,9618,0,0,0,0,0,0,18,17.75837,8.494489


In [83]:
_comm = (
    df_email_breach_expanded.query("ecategory=='Commercial'")
    .query("seriousbreach==1")
    .groupby("email")["present"]
    .sum()
    .pipe(lambda s: s.describe().to_frame().T)
    .astype(int)
    .assign(perc_at_least_1=100 * n_pwned_pols_sbreach_comm / n_comm_emails)
    .assign(perc_at_least_2=100 * n_multi_pwned_pols_sbreach_comm / n_comm_emails)
)
_comm

,count,mean,std,min,25%,50%,75%,max,perc_at_least_1,perc_at_least_2
present,2766,1,4,0,0,0,1,189,34.851772,21.113521


In [84]:
_tab = (
    pd.concat([_all, _govt, _comm], ignore_index=True)
    .assign(
        **{
            col: lambda df_, col=col: df_[col].round(0).astype(int)
            for col in ["min", "25%", "50%", "75%", "max"]
        }
    )
    .round(1)
    .assign(perc_at_least_1=lambda df_: df_["perc_at_least_1"].astype(str) + "\\%")
    .assign(
        count=lambda df_: df_["count"].round(0).astype(int).apply(lambda x: f"{x:,}")
    )
    .astype(str)
    .assign(cat=["All emails", "Government emails", "Personal emails"])
    .reorder_columns(["cat"])
)
pandas_to_tex(_tab, "../tables/pooled_pols_seriousbreach_number_summary", index=False)
display(_tab)
!cat ../tables/pooled_pols_seriousbreach_number_summary.tex

,cat,count,mean,std,min,25%,50%,75%,max,perc_at_least_1,perc_at_least_2
0,All emails,"12,384",0.5,2.2,0,0,0,0,189,21.6\%,11.3
1,Government emails,"9,618",0.0,0.0,0,0,0,0,18,17.8\%,8.5
2,Personal emails,"2,766",1.0,4.0,0,0,0,1,189,34.9\%,21.1


\midrule
All emails & 12,384 & 0.5 & 2.2 & 0 & 0 & 0 & 0 & 189 & 21.6\% & 11.3 \\
Government emails & 9,618 & 0.0 & 0.0 & 0 & 0 & 0 & 0 & 18 & 17.8\% & 8.5 \\
Personal emails & 2,766 & 1.0 & 4.0 & 0 & 0 & 0 & 1 & 189 & 34.9\% & 21.1 \\

### Tabulate: Compromised data types

In [85]:
df_pwnpol_datatype = (
    df_email_breach_expanded.query("present==True")
    # ===================================================
    # Filter data types and remove duplicate breach info
    .set_index("email")
    .iloc[:, 15:-3]
    .reset_index()
    .drop_duplicates(ignore_index=True)
    # ===================================================
    # Long by email-datatype
    .melt(id_vars=["email"], var_name="datatype", value_name="present")
    .groupby(["email", "datatype"])["present"]
    .sum()
    .reset_index()
    .assign(present=lambda df_: np.where(df_["present"] > 0, 1, 0))
    # ===================================================
    # Back to wide by email
    .pivot(index="email", columns="datatype", values="present")
    # ===================================================
    .T.sum(axis=1)
    .reset_index(name="count")
    .sort_values(["count", "datatype"], ascending=[False, True], ignore_index=True)
    .assign(percent=lambda df_: 100 * df_["count"] / n_pwned_pols)
    .assign(percent=lambda df_: df_["percent"].apply(lambda x: f"{round(x, 1)}\\%"))
    # ===================================================
    .assign(
        seriousbreach=lambda df_: df_["datatype"]
        .isin(LIST_SERIOUS_DATACLASSES)
        .map({True: r"\checkmark", False: ""})
    )
)
df_pwnpol_datatype

,datatype,count,percent,seriousbreach
0,n_dataclasses,4092,100.0\%,
1,Email addresses,4091,100.0\%,
2,Names,3480,85.0\%,
3,Phone numbers,3201,78.2\%,
4,Job titles,2842,69.5\%,
5,Physical addresses,2702,66.0\%,
6,Social media profiles,2645,64.6\%,
7,Passwords,2597,63.5\%,\checkmark
8,Geographic locations,2090,51.1\%,
9,Employers,1868,45.7\%,


In [86]:
df_datatype1 = (
    df_pwnpol_datatype.iloc[:20]
    .reset_index(drop=True)
    .assign(ix=range(1, 21))
    .astype(str)
    .reorder_columns(["ix"])
)
df_datatype2 = (
    df_pwnpol_datatype.iloc[20:40]
    .reset_index(drop=True)
    .assign(ix=range(21, 41))
    .astype(str)
    .reorder_columns(["ix"])
)
df_datatype3 = (
    df_pwnpol_datatype.iloc[40:60]
    .reset_index(drop=True)
    .assign(ix=range(41, 61))
    .astype(str)
    .reorder_columns(["ix"])
)
df2tex = pd.concat([df_datatype1, df_datatype2, df_datatype3], axis=1)
# print(df2tex.to_latex(na_rep="", index=False))
df2tex

,ix,datatype,count,percent,seriousbreach,ix,datatype,count,percent,seriousbreach,ix,datatype,count,percent,seriousbreach
0,1,n_dataclasses,4092,100.0\%,,21,Job applications,113,2.8\%,,41,Bank account numbers,40,1.0\%,\checkmark
1,2,Email addresses,4091,100.0\%,,22,Marital statuses,99,2.4\%,,42,Payment histories,38,0.9\%,
2,3,Names,3480,85.0\%,,23,Religions,99,2.4\%,,43,Survey results,34,0.8\%,
3,4,Phone numbers,3201,78.2\%,,24,Email messages,96,2.3\%,\checkmark,44,User website URLs,31,0.8\%,
4,5,Job titles,2842,69.5\%,,25,Password hints,81,2.0\%,\checkmark,45,Telecommunications carrier,30,0.7\%,
5,6,Physical addresses,2702,66.0\%,,26,Ethnicities,77,1.9\%,,46,Nationalities,27,0.7\%,\checkmark
6,7,Social media profiles,2645,64.6\%,,27,Home ownership statuses,77,1.9\%,,47,Private messages,27,0.7\%,\checkmark
7,8,Passwords,2597,63.5\%,\checkmark,28,Auth tokens,75,1.8\%,\checkmark,48,Relationship statuses,21,0.5\%,
8,9,Geographic locations,2090,51.1\%,,29,Occupations,75,1.8\%,,49,Company names,20,0.5\%,
9,10,Employers,1868,45.7\%,,30,PINs,69,1.7\%,\checkmark,50,Deceased statuses,18,0.4\%,


In [87]:
pandas_to_tex(
    df2tex, "../tables/hipb_pwnpols_datatypes.tex", na_rep="", index=False, escape=False
)

!cat "../tables/hipb_pwnpols_datatypes.tex"

\midrule
1 & n_dataclasses & 4092 & 100.0\% &  & 21 & Job applications & 113 & 2.8\% &  & 41 & Bank account numbers & 40 & 1.0\% & \checkmark \\
2 & Email addresses & 4091 & 100.0\% &  & 22 & Marital statuses & 99 & 2.4\% &  & 42 & Payment histories & 38 & 0.9\% &  \\
3 & Names & 3480 & 85.0\% &  & 23 & Religions & 99 & 2.4\% &  & 43 & Survey results & 34 & 0.8\% &  \\
4 & Phone numbers & 3201 & 78.2\% &  & 24 & Email messages & 96 & 2.3\% & \checkmark & 44 & User website URLs & 31 & 0.8\% &  \\
5 & Job titles & 2842 & 69.5\% &  & 25 & Password hints & 81 & 2.0\% & \checkmark & 45 & Telecommunications carrier & 30 & 0.7\% &  \\
6 & Physical addresses & 2702 & 66.0\% &  & 26 & Ethnicities & 77 & 1.9\% &  & 46 & Nationalities & 27 & 0.7\% & \checkmark \\
7 & Social media profiles & 2645 & 64.6\% &  & 27 & Home ownership statuses & 77 & 1.9\% &  & 47 & Private messages & 27 & 0.7\% & \checkmark \\
8 & Passwords & 2597 & 63.5\% & \checkmark & 28 & Auth tokens & 75 & 1.8\% & \checkmark & 48

### Tabulate: Breaches

In [88]:
df_pwnpol_breach_incident = (
    df_email_breach_expanded.query("present==True")
    # ===================================================
    .groupby(["breach"])["email"]
    .size()
    .reset_index()
    .rename_column("email", "emails")
    .sort_values("emails", ascending=False, ignore_index=True)
    .head(50)
    .assign(percent=lambda df_: 100 * df_["emails"] / n_pwned_pols)
    .assign(percent=lambda df_: df_["percent"].apply(lambda x: f"{round(x, 1)}\%"))
    # ===================================================
    # Merge back to get breach characteristics
    .merge(
        (
            pd.read_parquet("../data/breaches_01_2025_expanded.parquet")
            .remove_columns(
                [
                    "description",
                    "title",
                    "modifieddate",
                    "logopath",
                    "timetopublic",
                    "dataclasses",
                ]
            )
            .rename_column("name", "breach")
            .assign(
                seriousbreach=lambda df_: df_[LIST_SERIOUS_DATACLASSES]
                .max(axis=1)
                .astype(int)
            )
            .assign(
                seriousbreach=lambda df_: np.where(
                    df_["seriousbreach"] == 1, r"\checkmark", ""
                )
            )
            #             .filter(["breach", "breachdate", "addeddate", "yearstopublic", "n_dataclasses", "seriousbreach", *LIST_ALL_DATACLASSES])
            .filter(
                [
                    "breach",
                    "domain",
                    "breachdate",
                    "addeddate",
                    "yearstopublic",
                    "pwncount",
                    "n_dataclasses",
                    "seriousbreach",
                ]
            )
        ),
        how="left",
        on="breach",
        validate="1:1",
    )
    # ===================================================
    # Remove timestamps
    .assign(
        breachdate=lambda df_: pd.to_datetime(df_["breachdate"]).dt.date,
        addeddate=lambda df_: pd.to_datetime(df_["addeddate"]).dt.date,
    )
    .assign(pwncount=lambda df_: df_["pwncount"] / 1_000_000)
    .round(1)
    .astype(str)
    .assign(pwncount=lambda df_: df_["pwncount"] + "M")
    .assign(yearstopublic=lambda df_: df_["yearstopublic"] + " years")
    # ===================================================
    .assign(ix=lambda df_: range(1, 1 + len(df_)))
    .reorder_columns(["ix"])
    .replace("None", "---")
)
df_pwnpol_breach_incident

,ix,breach,emails,percent,domain,breachdate,addeddate,yearstopublic,pwncount,n_dataclasses,seriousbreach
0,1,db8151dd,1699,41.5\%,covve.com,2020-02-20,2020-05-15,0.2 years,22.8M,6,
1,2,OnlinerSpambot,1285,31.4\%,---,2017-08-28,2017-08-29,0.0 years,711.5M,2,\checkmark
2,3,PDL,1170,28.6\%,---,2019-10-16,2019-11-22,0.1 years,622.2M,7,
3,4,VerificationsIO,1078,26.3\%,verifications.io,2019-02-25,2019-03-09,0.0 years,763.1M,10,
4,5,LinkedIn,512,12.5\%,linkedin.com,2012-05-05,2016-05-21,4.0 years,164.6M,2,\checkmark
5,6,LinkedInScrape,398,9.7\%,linkedin.com,2021-04-08,2021-10-02,0.5 years,125.7M,7,
6,7,Apollo,346,8.5\%,apollo.io,2018-07-23,2018-10-05,0.2 years,125.9M,8,
7,8,Intelimost,294,7.2\%,intelimost.com,2019-03-10,2019-04-02,0.1 years,3.1M,2,\checkmark
8,9,Twitter200M,286,7.0\%,twitter.com,2021-01-01,2023-01-05,2.0 years,211.5M,4,
9,10,Cit0day,284,6.9\%,cit0day.in,2020-11-04,2020-11-19,0.0 years,226.9M,2,\checkmark


In [89]:
pandas_to_tex(
    df_pwnpol_breach_incident.head(25),
    "../tables/hipb_pwnpols_breach_incidents.tex",
    escape=False,
)

In [90]:
!cat ../tables/hipb_pwnpols_breach_incidents.tex

\midrule
1 & db8151dd & 1699 & 41.5\% & covve.com & 2020-02-20 & 2020-05-15 & 0.2 years & 22.8M & 6 &  \\
2 & OnlinerSpambot & 1285 & 31.4\% & --- & 2017-08-28 & 2017-08-29 & 0.0 years & 711.5M & 2 & \checkmark \\
3 & PDL & 1170 & 28.6\% & --- & 2019-10-16 & 2019-11-22 & 0.1 years & 622.2M & 7 &  \\
4 & VerificationsIO & 1078 & 26.3\% & verifications.io & 2019-02-25 & 2019-03-09 & 0.0 years & 763.1M & 10 &  \\
5 & LinkedIn & 512 & 12.5\% & linkedin.com & 2012-05-05 & 2016-05-21 & 4.0 years & 164.6M & 2 & \checkmark \\
6 & LinkedInScrape & 398 & 9.7\% & linkedin.com & 2021-04-08 & 2021-10-02 & 0.5 years & 125.7M & 7 &  \\
7 & Apollo & 346 & 8.5\% & apollo.io & 2018-07-23 & 2018-10-05 & 0.2 years & 125.9M & 8 &  \\
8 & Intelimost & 294 & 7.2\% & intelimost.com & 2019-03-10 & 2019-04-02 & 0.1 years & 3.1M & 2 & \checkmark \\
9 & Twitter200M & 286 & 7.0\% & twitter.com & 2021-01-01 & 2023-01-05 & 2.0 years & 211.5M & 4 &  \\
10 & Cit0day & 284 & 6.9\% & cit0day.in & 2020-11-04 & 2020-11-19